In [1]:
import pandas as pd
import re 
#import datetime as dt

In [2]:
df = pd.read_csv('incident_data.csv')

In [3]:
# typing, converting date to dt obj
df["date"] = pd.to_datetime(df["date"])

In [4]:
# Making type of officer that released report into a new column
def officer_type(officername):
    officer_prefixes = ["PIO", "Sgt.", "Lt.", "Capt.", "P.O."]
    split_name = officername.split(" ")
    prefix = split_name[0]
    if prefix in officer_prefixes:
        officer_type = prefix
    else:
        officer_type = "Unspecified"
    return officer_type

df["officer_type"] = df["officer"].apply(officer_type)
df["officer_type"] = df["officer_type"].str.replace("PIO", "Public Information Officer")
df["officer_type"] = df["officer_type"].str.replace("P.O.", "Police Officer")
df["officer_type"] = df["officer_type"].str.replace("Sgt.", "Sergeant")
df["officer_type"] = df["officer_type"].str.replace("Lt.", "Lieutenant")
df["officer_type"] = df["officer_type"].str.replace("Capt.", "Captain")

df["officer_type"].unique()

C:\Users\shane\AppData\Local\Temp\ipykernel_113436\3740180151.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df["officer_type"] = df["officer_type"].str.replace("P.O.", "Police Officer")
C:\Users\shane\AppData\Local\Temp\ipykernel_113436\3740180151.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df["officer_type"] = df["officer_type"].str.replace("Sgt.", "Sergeant")
C:\Users\shane\AppData\Local\Temp\ipykernel_113436\3740180151.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df["officer_type"] = df["officer_type"].str.replace("Lt.", "Lieutenant")
C:\Users\shane\AppData\Local\Temp\ipykernel_113436\3740180151.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df["officer_type"] = df["officer_type"].str.replace("Capt.", "Captain")


array(['Police Officer', 'Unspecified', 'Sergeant', 'Lieutenant',
       'Public Information Officer', 'Captain'], dtype=object)

In [5]:
# Parse Release Time from Description, remove last line with officer info

## SHOULD COMBINE THESE INTO FORMAT DESCRIPTION FUNCTION< REMOVE ALL BACKSLASHES



def parse_release(description):
    time_and_officer = description.split("Released ")[-1].split("at ")[-1]
    release_time = time_and_officer.split(" by")[0]
    return release_time
    
    
    
df["release_time"] = df["description"].apply(parse_release)


def remove_infoline(description):
    description = ' '.join(description.split("\n\n")[:-1])
    return description

df["description"] = df["description"].apply(remove_infoline)

df["description"]

0       On Sunday afternoon, Madison Police officers r...
1       Recently, there has been an uptick in scams ta...
2       Yesterday afternoon at approximately 2:00 p.m....
3       As part of the Madison Police Department's com...
4       Missing is a Theodore (TJ) Weege. TJ is a whit...
                              ...                        
2133    Two suspects were seen on surveillance video a...
2134    The MPD is looking to identify a man who spoke...
2135    On 11/26/19 at approximately 4:30pm, the 58 yo...
2136    At approximately 3:09PM a vehicle was stolen i...
2137     A downtown burglary and theft suspect was ide...
Name: description, Length: 2138, dtype: object

In [10]:
df["description"][1]

'Recently, there has been an uptick in scams targeting elderly victims. This increasing trend has been seen in our community and others, and has many different forms. These scams take many forms, but are typically seen when the intended victim receives a call from someone pretending to be their grandchild or a loved one. The "grandchild," in reality is the scammer, and often times will sound panicked and will be asking for help. The scammer will claim that they\'ve been in a car accident, have been arrested, or were the victim of some kind of crime. The scammer will then ask for money, usually with a demand that the victim deliver this immediately and not talk to anyone about the situation. We have seen situations lately where the scammers will ask for cash, and send a "courier," to collect the cash. Other circumstances will ask for bank or credit card information. MPD has also dealt with circumstances where the scammer asks for the victim to go to a Bitcoin ATM and complete a transact

In [6]:
# Remove NaN values
df["updated"]

0              NaN
1              NaN
2              NaN
3              NaN
4       11/07/2022
           ...    
2133           NaN
2134           NaN
2135    12/02/2019
2136           NaN
2137           NaN
Name: updated, Length: 2138, dtype: object

In [7]:
# Apply length to description
df["len_desc"] = df["description"].apply(len)
df.head()

,date,incident,address,officer,casenumber,updated,link,description,officer_type,release_time,len_desc
0,2022-11-14,Damaged Property,400 block S. Midvale Blvd.,P.O. Hunter Lisko,2022-464356,NaN,https://www.cityofmadison.com/police/newsroom/...,"On Sunday afternoon, Madison Police officers r...",Police Officer,10:32 AM,727
1,2022-11-09,Fraud,211 S. Carroll St.,P.O. Hunter Lisko,2022-000000,NaN,https://www.cityofmadison.com/police/newsroom/...,"Recently, there has been an uptick in scams ta...",Police Officer,2:12 PM,2493
2,2022-11-09,Traffic Incident,Beltline/S. Midvale Blvd.,P.O. Hunter Lisko,2022-457354,NaN,https://www.cityofmadison.com/police/newsroom/...,Yesterday afternoon at approximately 2:00 p.m....,Police Officer,8:55 AM,1153
3,2022-11-08,Traffic Incident,900 block E. Washington Ave.,P.O. Hunter Lisko,2022-277138,NaN,https://www.cityofmadison.com/police/newsroom/...,As part of the Madison Police Department's com...,Police Officer,2:03 PM,894
4,2022-11-03,Missing Adult,West Side,Livia Novitzke,2022-446354,11/07/2022,https://www.cityofmadison.com/police/newsroom/...,Missing is a Theodore (TJ) Weege. TJ is a whit...,Unspecified,8:21 PM,226


In [8]:
# TO DO:
# What to do with NaN in updated?
# Pull stuff from description ( parse? length)
# Most common keywords in description?
# Build function to lengthen name of PO/ make it a new column


# TYPING
# 	Hour of Day would be cool
# 	Geocoding
# 	Analysis of Decription,
# 	Description length
# 	Remove Information? Maybe (for plots and desc)
# 	Feature Engineering
# 	Df[“violent”] 0 or 1 based on incident type
# Df[“length_desc”]
# typ e all of the data, datetimes, 
# Possibly make Columns of value_counts of type, neighborhood,



# Possibly parse out the street? 
# Find KMLS or shapefiles for addresses
